In [1]:
import sqlite3

import timeit

import pandas

from data_algebra.data_ops import *
import data_algebra.SQLite
import data_algebra.test_util

In [2]:
d = pandas.read_csv('d.csv.gz')

d.head()

,g,v_00001,v_00002,v_00003,v_00004,v_00005,v_00006,v_00007,v_00008,v_00009,v_00010
0,level_000746732,0.501391,-0.042046,0.907852,1.103653,-0.933225,1.991693,-0.154361,-0.991363,2.328447,0.222225
1,level_000424470,0.512520,-0.536119,-0.827670,-1.587808,-0.047809,0.437973,-0.388076,-0.227378,0.454036,0.340655
2,level_000463169,-1.248967,-1.006886,0.352715,0.847306,1.417280,-1.852341,0.526444,0.051136,-0.053652,1.893562
3,level_000025764,1.093854,-0.975188,0.358386,0.381903,0.513759,0.710493,0.100913,-0.588443,-0.159640,-0.923627
4,level_000630470,0.319834,-0.073545,1.457324,-1.507512,-0.670575,-0.870075,-0.131663,-0.452909,1.415066,-2.134600


In [3]:
d.shape

(1000000, 11)

In [4]:
vars = [c for c in d.columns if not c == 'g']

ops = describe_table(d, table_name='d'). \
    extend({'max_' + v: v + '.max()' for v in vars},
        partition_by=['g']). \
    order_rows(['g'] + vars)

ops    

TableDescription(
 table_name='d',
 column_names=[
   'g', 'v_00001', 'v_00002', 'v_00003', 'v_00004', 'v_00005', 'v_00006',  
 'v_00007', 'v_00008', 'v_00009', 'v_00010']) .\
   extend({
    'max_v_00001': 'v_00001.max()',
    'max_v_00002': 'v_00002.max()',
    'max_v_00003': 'v_00003.max()',
    'max_v_00004': 'v_00004.max()',
    'max_v_00005': 'v_00005.max()',
    'max_v_00006': 'v_00006.max()',
    'max_v_00007': 'v_00007.max()',
    'max_v_00008': 'v_00008.max()',
    'max_v_00009': 'v_00009.max()',
    'max_v_00010': 'v_00010.max()'},
   partition_by=['g']) .\
   order_rows(['g', 'v_00001', 'v_00002', 'v_00003', 'v_00004', 'v_00005', 'v_00006', 'v_00007', 'v_00008', 'v_00009', 'v_00010'])

In [5]:
res = ops.transform(d)

res.head()

,g,v_00001,v_00002,v_00003,v_00004,v_00005,v_00006,v_00007,v_00008,v_00009,...,max_v_00001,max_v_00002,max_v_00003,max_v_00004,max_v_00005,max_v_00006,max_v_00007,max_v_00008,max_v_00009,max_v_00010
0,level_000000002,0.480053,0.556812,-0.295186,1.069603,-1.287380,-0.343787,-0.555874,0.481993,-0.085779,...,0.480053,0.556812,-0.295186,1.069603,-1.287380,-0.343787,-0.555874,0.481993,-0.085779,-1.203414
1,level_000000003,-0.052534,0.983563,0.145466,1.153262,-0.102269,0.593555,-0.437793,-0.052661,1.365170,...,-0.052534,0.983563,0.145466,1.153262,-0.102269,0.593555,-0.437793,-0.052661,1.365170,1.840541
2,level_000000004,0.114769,-0.228287,-0.739238,0.681996,-0.476465,-0.815794,0.426362,0.308667,-0.685185,...,1.302818,-0.020408,-0.591229,0.681996,0.031225,0.518879,0.426362,0.522919,0.031270,0.647587
3,level_000000004,1.302818,-0.020408,-0.591229,-0.453501,0.031225,0.518879,-0.724670,0.522919,0.031270,...,1.302818,-0.020408,-0.591229,0.681996,0.031225,0.518879,0.426362,0.522919,0.031270,0.647587
4,level_000000005,0.209939,0.568525,-0.657119,1.791830,1.800427,-0.123661,0.084579,0.057838,1.047468,...,1.017089,0.568525,-0.022681,1.791830,1.800427,0.519874,0.084579,1.805242,1.047468,2.604739


In [6]:
expect = pandas.read_csv('res.csv.gz')

expect.head()

,g,v_00001,v_00002,v_00003,v_00004,v_00005,v_00006,v_00007,v_00008,v_00009,...,max_v_00001,max_v_00002,max_v_00003,max_v_00004,max_v_00005,max_v_00006,max_v_00007,max_v_00008,max_v_00009,max_v_00010
0,level_000000002,0.480053,0.556812,-0.295186,1.069603,-1.287380,-0.343787,-0.555874,0.481993,-0.085779,...,0.480053,0.556812,-0.295186,1.069603,-1.287380,-0.343787,-0.555874,0.481993,-0.085779,-1.203414
1,level_000000003,-0.052534,0.983563,0.145466,1.153262,-0.102269,0.593555,-0.437793,-0.052661,1.365170,...,-0.052534,0.983563,0.145466,1.153262,-0.102269,0.593555,-0.437793,-0.052661,1.365170,1.840541
2,level_000000004,0.114769,-0.228287,-0.739238,0.681996,-0.476465,-0.815794,0.426362,0.308667,-0.685185,...,1.302818,-0.020408,-0.591229,0.681996,0.031225,0.518879,0.426362,0.522919,0.031270,0.647587
3,level_000000004,1.302818,-0.020408,-0.591229,-0.453501,0.031225,0.518879,-0.724670,0.522919,0.031270,...,1.302818,-0.020408,-0.591229,0.681996,0.031225,0.518879,0.426362,0.522919,0.031270,0.647587
4,level_000000005,0.209939,0.568525,-0.657119,1.791830,1.800427,-0.123661,0.084579,0.057838,1.047468,...,1.017089,0.568525,-0.022681,1.791830,1.800427,0.519874,0.084579,1.805242,1.047468,2.604739


In [7]:
assert data_algebra.test_util.equivalent_frames(res, expect)

In [8]:


reps = 5

def f():
    return ops.transform(d)

time_pandas = timeit.timeit(f, number=reps)
time_pandas

162.44369801099998

In [9]:
time_pandas/reps

32.4887396022

In [10]:
dbmodel = data_algebra.SQLite.SQLiteModel()

In [11]:
print(ops.to_sql(dbmodel, pretty=True))

SELECT "g",
       "v_00001",
       "v_00002",
       "v_00003",
       "v_00004",
       "v_00005",
       "v_00006",
       "v_00007",
       "v_00008",
       "v_00009",
       "v_00010",
       "max_v_00001",
       "max_v_00002",
       "max_v_00003",
       "max_v_00004",
       "max_v_00005",
       "max_v_00006",
       "max_v_00007",
       "max_v_00008",
       "max_v_00009",
       "max_v_00010"
FROM
  (SELECT "g",
          "v_00001",
          "v_00002",
          "v_00003",
          "v_00004",
          "v_00005",
          "v_00006",
          "v_00007",
          "v_00008",
          "v_00009",
          "v_00010",
          MAX("v_00001") OVER (PARTITION BY "g") AS "max_v_00001",
                              MAX("v_00002") OVER (PARTITION BY "g") AS "max_v_00002",
                                                  MAX("v_00003") OVER (PARTITION BY "g") AS "max_v_00003",
                                                                      MAX("v_00004") OVER (PARTITI

In [12]:
conn = sqlite3.connect(':memory:')
dbmodel.prepare_connection(conn)

In [13]:
def f_db():
    try:
        dbmodel.read_query(conn, "DROP TABLE d")
    except:
        pass
    dbmodel.insert_table(conn, d, 'd')
    return dbmodel.read_query(conn, ops.to_sql(dbmodel))

In [14]:
res_db = f_db()

assert data_algebra.test_util.equivalent_frames(res_db, expect)

In [15]:
time_sql = timeit.timeit(f_db, number=reps)
time_sql

147.825360993

In [16]:
time_sql/reps

29.5650721986

In [17]:
# neaten up
conn.close()